# NTHA Recall Analysis

The purpose of this analysis is to summarize and better understand recall trends. This will help prepare for service disruptions caused by recall campaigns.
Please see the summary of findings for an analysis of the data. The summary also includes steps for future analysis and modeling. The code and
visualizations are provided here.

Variable Name|Description
--- | ---
RECORD_ID|RUNNING SEQUENCE NUMBER, WHICH UNIQUELY IDENTIFIES THE RECORD.
CAMPNO|NHTSA CAMPAIGN NUMBER
MAKETXT|VEHICLE/EQUIPMENT MAKE
MODELTXT|VEHICLE/EQUIPMENT MODEL
YEARTXT|MODEL YEAR, 9999 IF UNKNOWN or NaN
MFGCAMPNO|MFR CAMPAIGN NUMBER
COMPNAME|COMPONENT DESCRIPTION
MFGNAME|MANUFACTURER THAT FILED DEFECT/NONCOMPLIANCE REPORT
BGMAN|BEGIN DATE OF MANUFACTURING
ENDMAN|END DATE OF MANUFACTURING
RCLTYPECD|VEHICLE, EQUIPMENT OR TIRE REPORT
POTAFF|POTENTIAL NUMBER OF UNITS AFFECTED
ODATE|DATE OWNER NOTIFIED BY MFR
INFLUENCED_BY|RECALL INITIATOR (MFR/OVSC/ODI)
MFGTXT|MANUFACTURERS OF RECALLED VEHICLES/PRODUCTS
RCDATE|REPORT RECEIVED DATE
DATEA|RECORD CREATION DATE
RPNO|REGULATION PART NUMBER
FMVSS|FEDERAL MOTOR VEHICLE SAFETY STANDARD NUMBER
DESC_DEFECT|DEFECT SUMMARY
CONEQUENCE_DEFECT|CONSEQUENCE SUMMARY
CORRECTIVE_ACTION|CORRECTIVE SUMMARY
NOTES|RECALL NOTES
RCL_CMPT_ID|NUMBER THAT UNIQUELY IDENTIFIES A RECALLED COMPONENT.
MFR_COMP_NAME|MANUFACTURER-SUPPLIED COMPONENT NAME
MFR_COMP_DESC|MANUFACTURER-SUPPLIED COMPONENT DESCRIPTION
MFR_COMP_PTNO|MANUFACTURER-SUPPLIED COMPONENT PART NUMBER

In [1]:
from datetime import datetime as dt

import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

pd.set_option('max_columns', None)
# plt.rcParams('figure.figsize'
# plt.rcParams('font.size') = 1

# import data, using correct dtypes
datecols = [8,9,12,15,16]

file = '../input/vehicle-recall-data/FLAT_RCL.txt'
raw = pd.read_csv(file, sep = '\t',encoding = 'latin1',names=list(range(27)), low_memory=False, parse_dates=datecols)

print('Raw data dimensions: ',raw.shape, '\n')

Raw data dimensions:  (228580, 27) 



In [2]:
manufacturers = ['BMW','MINI','MERCEDES','LEXUS','NISSAN','AUDI','VOLKSWAGEN','HYUNDAI','INFINITI','KIA','MAZDA',
                'LAND ROVER','VOLVO','HONDA','JAGUAR','ACURA','TOYOTA','MOPAR','SUBARU','PORSCHE']

names = ['RECORD_ID','CAMPNO','MAKETXT','MODELTXT','YEARTXT','MFGCAMPNO','COMPNAME','MFGNAME','BGMAN','ENDMAN','RCLTYPECD',
        'POTAFF','ODATE','INFLUENCED_BY','MFGTXT','RCDATE','DATEA','RPNO','FMVSS','DESC_DEFECT','CONEQUENCE_DEFECT',
        'CORRECTIVE_ACTION','NOTES','RCL_CMPT_ID','MFR_COMP_NAME','MFR_COMP_DESC','MFR_COMP_PTNO']

# keep only relevant car manufacturers
data = raw[raw[2].isin(manufacturers)]

#rename columms 
data.columns = names

# drop 1 record in POTAFF column
data.dropna(subset=['POTAFF'], axis=0, inplace=True)

# inspect the data
print('Relevant data dimensions: ', data.shape, '\n')
print(data.info())
data.head(n=3)

Relevant data dimensions:  (32631, 27) 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32631 entries, 48 to 228406
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   RECORD_ID          32631 non-null  int64         
 1   CAMPNO             32631 non-null  object        
 2   MAKETXT            32631 non-null  object        
 3   MODELTXT           32631 non-null  object        
 4   YEARTXT            32631 non-null  int64         
 5   MFGCAMPNO          20815 non-null  object        
 6   COMPNAME           32631 non-null  object        
 7   MFGNAME            32631 non-null  object        
 8   BGMAN              15022 non-null  object        
 9   ENDMAN             15074 non-null  object        
 10  RCLTYPECD          32631 non-null  object        
 11  POTAFF             32631 non-null  float64       
 12  ODATE              31358 non-null  object        
 13  INFLUENCED_BY     

/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,RECORD_ID,CAMPNO,MAKETXT,MODELTXT,YEARTXT,MFGCAMPNO,COMPNAME,MFGNAME,BGMAN,ENDMAN,RCLTYPECD,POTAFF,ODATE,INFLUENCED_BY,MFGTXT,RCDATE,DATEA,RPNO,FMVSS,DESC_DEFECT,CONEQUENCE_DEFECT,CORRECTIVE_ACTION,NOTES,RCL_CMPT_ID,MFR_COMP_NAME,MFR_COMP_DESC,MFR_COMP_PTNO
48,49,02V250000,HONDA,FSC600,2002,NaN,WHEELS,AMERICAN HONDA MOTOR CO.,20010808,20020708,V,2454.0,20020927,MFR,Honda (American Honda Motor Co.),2002-09-17,2002-09-17,NaN,NaN,"ON CERTAIN MOTORCYCLES, UNDER CERTAIN RIDING C...","THE SPOKES WILL EVENTUALLY FAIL, CAUSING THE W...","DEALERS WILL REPLACE THE REAR WHEEL, INCLUDING...","ALSO, CUSTOMERS CAN CONTACT THE NATIONAL HIGHW...",000015234000106390000000309,NaN,NaN,NaN
49,50,02V250000,HONDA,SILVER WING,2002,NaN,WHEELS,AMERICAN HONDA MOTOR CO.,20010808,20020708,V,2454.0,20020927,MFR,Honda (American Honda Motor Co.),2002-09-17,2002-09-17,NaN,NaN,"ON CERTAIN MOTORCYCLES, UNDER CERTAIN RIDING C...","THE SPOKES WILL EVENTUALLY FAIL, CAUSING THE W...","DEALERS WILL REPLACE THE REAR WHEEL, INCLUDING...","ALSO, CUSTOMERS CAN CONTACT THE NATIONAL HIGHW...",000015234000106389000000309,NaN,NaN,NaN
67,68,06V043000,VOLKSWAGEN,BEETLE,2006,M8,EQUIPMENT:OTHER:LABELS,"VOLKSWAGEN OF AMERICA, INC",20051004,20060128,V,906.0,20060227,MFR,"Volkswagen Group of America, Inc.",2006-02-09,2006-02-14,571,110,CERTAIN PASSENGER VEHICLES EQUIPPED WITH 17-IN...,A MISPRINTED LABEL COULD LEAD TO FITMENT OF I...,OWNERS WILL BE PROVIDED WITH A NEW TIRE INFORM...,VOLKSWAGEN RECALL NO. M8.CUSTOMERS MAY ALSO CO...,000022568000222738000000349,NaN,NaN,NaN


In [3]:
# total campaigns
total_camps = data['CAMPNO'].nunique()

#total cars affected in all camps
total_cars = data.groupby('CAMPNO')['POTAFF'].mean().sum()

print("There are {total_camps} recalls potentially affecting {total_cars} vehicles in this dataset.".format(total_camps=total_camps, total_cars=int(total_cars)))

There are 3730 recalls potentially affecting 307395126 vehicles in this dataset.


In [4]:
# total campaigns by mfr
total_camps_mfr = data.groupby('MAKETXT')['CAMPNO'].nunique().sort_values(ascending=False)

px.bar(x=total_camps_mfr.index, y=total_camps_mfr, title='Figure 1: Recalls Issued by Each Manufacturer',
      labels={'x':'', 'y':'# of Recalls'})

In [5]:
# total cars affected by each manufacturer
total_cars_mfr = data.groupby(['MAKETXT','CAMPNO'])['POTAFF'].mean().groupby('MAKETXT').sum().sort_values(ascending=False)

px.bar(x=total_cars_mfr.index, y=total_cars_mfr, title='Figure 2: Potential Vehicles affected by Manufacturer',
      labels={'x':'', 'y':'# of Vehicles'})

In [6]:
# group yearly trends
yearly_camps_mfr = data.groupby([data.DATEA.dt.year, data.MAKETXT])['CAMPNO'].nunique()

# plot yearly trends
px.line(yearly_camps_mfr.unstack(level=1)[1:], title='Figure 3: Yearly Recalls by Manufacturer', labels={'DATEA':'', 'value':'# of Recalls'})